## Example deployment function

Using global data

In [1]:
import pandas as pd

In [2]:
def dissonance(
    resp_dict,
    model_paths,
    classifier,
):
    """_summary_

    Args:
        resp_dict (_type_): response dictionary with keys as questions and values as subjects' responses
        model_paths (_type_): paths to qnet models used for computing dissonances
        question_order (_type_): order of questions used by qnets specified in `model_paths`
        classifier (_type_, optional): pretrained classifier used to compute malingering probability. Defaults to None.

    Returns:
        _type_: dissonance(s) of subjects (and malingering prob, if classifier is provided)
    """
    import numpy as np
    from quasinet.qnet import load_qnet

    def _diss_linear(s, qnet, missing_value=0):
        diss = list()
        Ds = qnet.predict_distributions(s)

        for i in range(len(s)):
            if s[i] != "":
                if s[i] in Ds[i].keys():
                    diss.append(1 - Ds[i][s[i]] / np.max(list(Ds[i].values())))
                elif s[i] == "missing":
                    diss.append(missing_value)
                else:
                    diss.append(1)

        return np.array(diss)

    def _actual_sample_dissonance(
        data_sample, diss_models, diss_fcn, order, length, missing_value=0
    ):
        if order is None:
            order = range(length)

        sample = np.full(length, "", dtype="<U21")

        diss = [list() for model in diss_models]

        for i in order:
            if data_sample[i] == "":
                sample[i] = "missing"
            else:
                sample[i] = data_sample[i]
            for d, model in zip(diss, diss_models):
                d.append(diss_fcn(sample, model, missing_value))

        return sample, diss

    def _all_actual_samples_dissonance(
        data_samples, diss_models, diss_fcn, order, length, missing_value=0
    ):
        samples = list()
        dissonances = list()

        for data_sample in data_samples:
            samp, diss = _actual_sample_dissonance(
                data_sample, diss_models, diss_fcn, order, length, missing_value
            )
            samples.append(samp)
            dissonances.append(diss)

        return samples, dissonances

    def _dissonance_data_at_question(dissonances, questions_asked):
        return np.array(
            [np.hstack([d[questions_asked - 1] for d in diss]) for diss in dissonances]
        )

    diss_models = [
        load_qnet(
            model_path,
            gz=True,
        )
        for model_path in model_paths
    ]
    resp_df = pd.DataFrame(resp_dict).T

    resp_fmtd = (
        resp_df[diss_models[0].feature_names]
        .fillna(-9)
        .astype(int)
        .replace(-9, "")
        .to_numpy()
    )

    resp_diss = _all_actual_samples_dissonance(
        resp_fmtd,
        diss_models,
        _diss_linear,
        range(resp_fmtd.shape[1]),
        resp_fmtd.shape[1],
    )[1]

    resp_new = pd.DataFrame(_dissonance_data_at_question(resp_diss, resp_fmtd.shape[1]))

    proba = classifier.predict_proba(resp_new)

    output_dict = dict.fromkeys(
        resp_dict.keys(),
    )
    for key, val in zip(output_dict.keys(), proba[:, 1]):
        output_dict[key] = val

    return output_dict

In [3]:
resp_dict = pd.read_pickle("global-data-example-dict.pkl")
model_paths = [
    f"models/global/random_order_{model}_model_0.joblib.gz"
    for model in ["full"]  # , "neg", "pos"]
]
classifier = pd.read_pickle(f"classifiers/global/runif-{346}.pkl")

In [4]:
dissonance(resp_dict, model_paths, classifier)

{0: 0.01,
 1: 0.0,
 2: 0.0,
 3: 0.0,
 4: 0.0,
 5: 0.02,
 6: 0.01,
 7: 0.0,
 8: 0.0,
 9: 0.04}